In [10]:
import os
import glob
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import lr_scheduler
import operator
import json
from sklearn import tree
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from collections import Counter
from sklearn.preprocessing import LabelEncoder, scale, PolynomialFeatures
from sklearn.datasets import load_boston
from sklearn.cross_decomposition import PLSRegression, PLSSVD
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, roc_auc_score, accuracy_score, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from datetime import datetime
from scipy import stats
import torch.utils.data
from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline, FeatureUnion
from torch.autograd import Variable

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [2]:
train_csv = pd.read_csv("Dataset/train.csv", keep_default_na=False)
test_csv = pd.read_csv("Dataset/test.csv", keep_default_na=False)

data = {}
labels = {}

In [3]:
train_csv.shape

(5959, 3)

In [4]:
def preprocess_data(dataset):
    dataset['Review Text'] = dataset["Review Title"].map(str) + " " + dataset['Review Text']    
    dataset = dataset.drop(['Review Title'],axis=1)
    return dataset

In [5]:
train = preprocess_data(train_csv)
test = preprocess_data(test_csv)

train_X = train["Review Text"]
train_y = train["topic"]
test_X = test["Review Text"]

#train_X, test_X, train_y, test_y = train_test_split(train["Review Text"], train["topic"], random_state=33)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))
#s = " ".join(train["Review Text"][0:10])
#print(s)

IMDb reviews (combined): train = 5959, test = 2553


In [6]:
import numpy as np
import re
from collections import Counter

def build_dict(data, vocab_size = 6300):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    word_counts = Counter(np.concatenate( data, axis=0 ))
    # word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
    print(len(sorted_words))

    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [7]:
#word_dict = build_dict(train_X)
# print(word_dict)
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    #words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

In [8]:
print(train_X[0])
review_to_words(train_X[0])

Useless Did nothing for me, didn't help lost even with working out and eating healthy. Didn't curb appetite or anything.


['useless',
 'nothing',
 'help',
 'lost',
 'even',
 'working',
 'eating',
 'healthy',
 'curb',
 'appetite',
 'anything']

In [9]:
import pickle

cache_dir = os.path.join("cache", "key_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    print(cache_file)
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            print("File not found")
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        print('Preprocess training and test data to obtain words for each review')
        # Preprocess training and test data to obtain words for each review
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        print('Preprocess training and test data to obtain words for each review')
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        print('Unpack data loaded from cache file')
        # Unpack data loaded from cache file
        words_train, words_test, labels_train = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'])
    
    return words_train, words_test, labels_train

In [10]:
# Preprocess data
train_X, test_X, train_y = preprocess_data(train_X, test_X, train_y)

preprocessed_data.pkl
Read preprocessed data from cache file: preprocessed_data.pkl
Unpack data loaded from cache file


In [11]:
word_dict = build_dict(train_X)

6268


In [12]:
# determine the five most frequently appearing words in the training set.
word_counts = Counter(np.concatenate( train_X, axis=0 ))
sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
print(sorted_words[0:5])
print(word_counts['tast'])

['tast', 'product', 'like', 'flavor', 'brand']
3109


In [13]:
#word_dict

In [14]:
with open('word_dict.json', 'w') as f:
    json.dump(word_dict, f)

In [15]:
with open(os.path.join(cache_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

In [16]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [17]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [18]:
print(len(train_X[0]))
print(train_X_len[0:1]) # 500 - 322 (non zeros) = 178

500
[11]


In [ ]:
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1).to_csv('sample_train.csv', header=False, index=False)

In [260]:
class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [261]:
import torch.nn as nn

class DNNClassifier(nn.Module):
    """
    This is the simple DNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, hidden_dim, input_size, output_size, dropout=0.5):
        """
        Initialize the model by settingg up the various layers.
        """
        super(DNNClassifier, self).__init__()

        self.sig = nn.Sigmoid()        
        # self.word_dict = None
        
        self.fc1 = nn.Linear(input_size, hidden_dim * 4)
        self.fc2 = nn.Linear(hidden_dim * 4, hidden_dim * 2)
        self.fc3 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_size)
        self.dropout = nn.Dropout(p=0.5)
        self.init_weights()
        self.soft = nn.LogSoftmax(dim=0)
        
    def init_weights(m):
        initrange = 0.08
        classname = m.__class__.__name__
        if classname.find('Linear') != -1:
            # get the number of the inputs
            n = m.in_features
            y = 1.0/np.sqrt(n)
            m.weight.data.normal_(0.0, y)
            m.bias.data.fill_(0)
        
    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        out = self.out(x)
        return self.soft(out.squeeze())

In [269]:
# Read in only the first 250 rows
train_sample = pd.read_csv('sample_train.csv', header=None, names=None)

lbl = LabelEncoder() 
lbl.fit(list(train_sample[[0]].values)) 

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(lbl.transform(list(train_sample[[0]].values))).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

In [270]:
# Hyperparameters
batch_size = 50
num_epochs = 10
# Learning Rate
learning_rate = 0.001
# Model parameters
# Input size
input_size = train_sample_X.shape[1]
# Output size
output_size = 1
# Embedding Dimension
embedding_dim = 32
# Hidden Dimension
hidden_dim = 200
# Number of RNN Layers
n_layers = 2
#vocabulary size
vocab_size = 6300

# Show stats for every n number of batches
show_every_n_batches = 50

In [271]:
# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=batch_size)

In [272]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # model.zero_grad()
            optimizer.zero_grad()
            
            #batch_len = batch_X.t()[0,:]
            #print(batch_len)
            
            # TODO: Complete this train method to train the model provided.
            output = model.forward(batch_X)
            #print(output)
            #print(output.shape)
            #print(output.permute(1, 21, batch_size))
            
            #print(batch_y.uniform_(0, 5).long())
            #print(batch_y.reshape(-1,1))
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))
        total_loss = 0
    return model

In [273]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [274]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = DNNClassifier(hidden_dim, input_size, output_size).to(device)
model = LSTMClassifier(embedding_dim, hidden_dim, vocab_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.NLLLoss()
loss_fn = torch.nn.BCELoss()

model_tn = train(model, train_sample_dl, num_epochs, optimizer, loss_fn, device)

Epoch: 1, BCELoss: -141.7180613403519
Epoch: 2, BCELoss: -215.15137875874836
Epoch: 3, BCELoss: -216.5000442504883
Epoch: 4, BCELoss: -216.98805720011393
Epoch: 5, BCELoss: -217.33083788553873
Epoch: 6, BCELoss: -217.5876350402832
Epoch: 7, BCELoss: -217.76557706197102
Epoch: 8, BCELoss: -217.8358247121175


KeyboardInterrupt: 

In [ ]:
with open('save/trained_rnn_new', 'wb') as pickle_file:
    torch.save(model_tn, pickle_file)

In [83]:
sequence_length   = 75
number_of_classes = 55
# creates random tensor of your output shape
output = torch.rand(sequence_length, 1, number_of_classes)
# creates tensor with random targets
target = torch.randint(55, (75,)).long()

# define loss function and calculate loss
criterion = nn.CrossEntropyLoss()
print(output)
print(target)
# apply squeeze() on output tensor to change shape form [75, 1, 55] to [75, 55]
loss = criterion(output.squeeze(), target)
print(loss)

tensor([[[0.4091, 0.0571, 0.5403,  ..., 0.0477, 0.5953, 0.3135]],

        [[0.0628, 0.6934, 0.8211,  ..., 0.9674, 0.8123, 0.6410]],

        [[0.1094, 0.4591, 0.0050,  ..., 0.9556, 0.8776, 0.8520]],

        ...,

        [[0.9357, 0.5406, 0.7774,  ..., 0.4005, 0.0547, 0.0434]],

        [[0.7748, 0.1234, 0.5048,  ..., 0.8279, 0.0447, 0.0692]],

        [[0.6944, 0.4479, 0.9415,  ..., 0.2698, 0.5920, 0.4917]]])
tensor([19, 13, 12, 30, 35, 35, 25,  4, 29,  0, 49, 37,  5,  5, 23, 18, 11, 18,
        36, 16, 21, 43,  3, 23, 52, 29, 21,  1, 34, 35, 37, 41, 13, 41, 11, 23,
        52, 10, 34, 38, 48, 52, 27, 36, 43, 26, 17,  9, 21, 53, 31, 26, 34, 26,
        20, 44, 51, 16, 26, 12, 40, 36, 16, 54, 14, 48, 10,  8,  1,  2, 10, 54,
         1, 28, 32])
tensor(4.0021)
